<h1 style="color:blue">Praktikum 2. Sinasõprus Pythoniga</h1>
<h3 style="color:blue">Kodeeringud, profileerimine, optimeerimine, kasulikud standardteegid</h3>
<h4>13. september 2016 </h4>
<h4>Ülesannete esitamise tähtaeg 27. september</h4>

Selles praktikumis tegeleme jätkuvalt suhteliselt tehniliste teemadega, kuid loomuliku keele töötluse puhul ei saa nendest üle ega ümber. 

## Kodeeringud
Loomuliku keele töötluse puhul - eriti, kui tegeleda mõne muu, mitte inglise keelega - tuleb tähelepanu pöörata ka kodeeringutele. Ei ole ju **kapp**, **käpp** ja **k�pp** samatähenduslikud või vähemalt mitte samavõrd üheselt mõistetava tähendusega, silmailust rääkimata. Samuti ei taha vale kodeeringuga töötamine hästi sujuda - selle kohta järgmine näide.

In [ ]:
fin = open("juku.txt", "r")
fin.readlines()

Proovides kõige tavalisemal moel failist Juku lugu sisse lugeda, saame veateate. 
Tõenäoliselt on meie vaikekodeeringuks UTF-8, Juku fail on aga ilmselt salvestatud mõnes muus. Kui me teame, mis kodeeringus fail on, siis suudab Python seda lugeda - selles on abiks teek **codecs**.

In [ ]:
# Oma vaikekodeeringu kontrollimine
import locale
locale.getpreferredencoding()

## Ülesanne 1. Kodeeringud (1p)
**a) Lugeda sisse fail "juku.txt", teades, et see on salvestatud kodeeringus WINDOWS-1257. Salvestada sama fail UTF-8's. (0,5p)**

**b) Üldistada tehtut: kirjutada funktsioon, mis saab argumentidena ette failinime, algse kodeeringu, kodeeringu, millesse fail konverteerida ning uue failinime. Funktsioon salvestab etteantud faili uues kodeeringus. (0.5p)**

In [ ]:
import codecs

In [ ]:
with codecs.open("juku.txt", "r", encoding = "WINDOWS-1257") as fin:
    # ...

In [ ]:
def encoding_converter(file1, enc1, enc2, file2):
    # ...
    return "File " + file1 + " saved as " + file2 + " in encoding " + enc2

## Ülesanne 2.  Hea ja halb kood (2p)

### Mantra: 
* töötav kood >>>>>>> optimaalne kood,
* optimeerimine on vaevanõudev,
* optimeeritud kood on raskemini loetav
* optimeeri siis, kui keegi ei pea lugema või optimaalsus on hädavajalik
* pikk kood ei pruugi olla aeglane ega kompaktne kood kiire

**Kiiruse suhtes optimaalne kood:**
* suhtleb minimaalselt kettal olevate failidega,
* suhtleb veel vähem veebis olevate failidega,
* hoiab võimalikult palju relevantseid asju mälus (muutujates),
* ei sisalda tarbetuid käske, mis arvutuse kulgu ei mõjuta,
* sisaldab probleemi kontekstis mõistliku keerukusega algoritmi,
* sisaldab probleemi kontekstis õigeid andmestruktuure
     * elementide kiireks hoiustamiseks list,
     * elementide kiireks otsimiseks sõnaraamat,
     * elementide kattuvuste vms kontrollimiseks hulk,
* arvutab ühte saadust vaid ühe korra,
* sisaldab minimaalselt funktsioonide kutseid,
* sisaldab minimaalselt tingimuslausete evalueerimist,
* ei eemalda elemente listi keskelt ega viska sinna elemente

**Järgnevalt on toodud funktsioon calculate_relative_freqs, mis saab sisendiks tee failini, mille igal real on
{sõna} {absoluutne sagedus dokumendis täisarvuna}
ning mis arvutab iga sõna jaoks selle sõna suhtelise sageduse dokumendis ja tagastab tulemuse dictionary'na.**

<h5 style="color:red">NB! Siin ülesandes on tegu on näitega halvast koodist, millest ei tasu mingil juhul eeskuju võtta!</h5>

**a) Vaadake koodi ja kirjeldage, mis muudab selle ebaefektiivseks (võite kirjutada vastavate koodiridade juurde kommentaaridena) (0,5p)**

In [ ]:
def calculate_relative_freqs(input_file):
    
    result = {}
    
    # Find all words for which there are frequencies
    with open(input_file, 'r') as fin:
        words = []
        for line in fin:
            words.append(line.strip().split()[1])

    # Find relative frequency for each word
    freqs = []

    for word in words:
        with open(input_file,'r') as fin:
            total_freq = 0
            word_freq = 0
            for line in fin:
                freq, word_candidate = line.strip().split()
                freqs.append(int(freq))

                if word == word_candidate:
                    word_freq += int(freq)

                total_freq += int(freq)

        freqs.sort()
        result[word] = float(word_freq) / total_freq
    return result 

**b) Kontrollige, kas teie oletustel oli tõepõhi all: selleks profileerige toodud funktsiooni. Väga lihtsasti loetava tulemuse annab tööriist line_profiler, mis on küll vaja eraldi conda kaudu installida (vt loenguslaidid). Muidugi võib kasutada ka standardteeke (profile, cProfile). Millised aegluse kohta püstitatud hüpoteesidest paika pidasid? (0,5p)**

Katsetamiseks on praktikumimaterjalidesse lisatud fail nimega **lemma_kahanevas.txt**. Tegu on eesti keele sagedussõnastikuga, mis on koostatud tasakaalus korpuse pealt (vt siit http://www.cl.ut.ee/ressursid/sagedused1/).
Failis on toodud sõnade absoluutsed sagedused, et aga hinnata, kui suure osa keelest tegelikult üks või teine sõna moodustab, oleks vaja teada suhtelisi sagedusi. Seda arvutabki eeltoodud funktsioon calculate_relative_freqs.

NB! Kuna toodud koodijupp on väga ebaefektiivne, siis on soovitav profileerimiseks ja muudeks katsetusteks kasutada esialgu vaid mingit osa (nt 500 esimest rida) antud failist.

**c) Muutke kood efektiivsemaks - parandage olemasolevat või soovi korral kirjutage uus ja parem funktsioon. Kontrollige profileerides, kas sai kiirem. (1p)**

## Ülesanne 3. Debugimine (1p) [PUUDUB]

## Ülesanne 4. Kasulikud standardteegid (1p)

Siin on mõningad näited teekidest, mis muudavad loomuliku keele töötleja elu mugavamaks.

**Defaultdict** - sagedusloendite tegemine

In [12]:
from collections import defaultdict

word = "habemeajaja"
freqs = defaultdict(int)

for char in word:
    freqs[char] += 1
    
freqs.items()    

dict_items([('h', 1), ('j', 2), ('e', 2), ('b', 1), ('m', 1), ('a', 4)])

**OrderedDict** - sõnastik, mis säilitab võti-väärtus-paaride järjekorra

In [30]:
from collections import OrderedDict

sorted_by_key = OrderedDict(sorted(freqs.items(), key=lambda t: t[0]))
sorted_by_value = OrderedDict(sorted(freqs.items(), key=lambda t: t[1], reverse = True))

In [31]:
sorted_by_key

OrderedDict([('a', 4), ('b', 1), ('e', 2), ('h', 1), ('j', 2), ('m', 1)])

In [32]:
sorted_by_value

OrderedDict([('a', 4), ('j', 2), ('e', 2), ('h', 1), ('b', 1), ('m', 1)])

**String.Template** ja **str.format** - lihtsad asendused sõnede sees

In [106]:
from string import Template

s = Template ("$kes tahab $mida")
s.substitute(kes = "Toomas", mida = "maja")

'Toomas tahab maja'

In [43]:
from datetime import datetime

now = datetime.now()
"Palju õnne tädi Maalile {0}. sünnipäevaks".format(now.year-1932)

'Palju õnne tädi Maalile 84. sünnipäevaks'

**str.translate** - kindlate tähemärkide asendamine

In [48]:
intab = "aeiouüõöä"
outtab = "uuuuuuuuu"
trantab = str.maketrans(intab, outtab)

str = "Põdral maja metsa sees"
str.translate(trantab)

'Pudrul muju mutsu suus'

### **re** - regulaaravaldised

<h4 style="color:green"> Regulaaravaldised on keeleandmete töötlemisel asendamatud. Kui keegi nendega tõesti veel kokku puutunud ei ole, siis on viimane aeg tutvuda https://docs.python.org/3/library/re.html ning eesti keeles https://courses.cs.ut.ee/2015/progmaa/Arhiiv/PARTVIIRegulaaravaldis1 </h4>

Regulaaravaldiste kasutamine ei piirdu Pythoni ega üleüldiselt programmeerimiskeeltega - neid on võimalik tekstis otsingute ja asenduste tegemiseks edukalt kasutada Linuxi käsurealt ('grep' käsuna), samuti mitmetes tekstitöötlusprogrammides alates MS Wordist ja lõpetades Notepad++ ja gedit'iga. 

In [56]:
import re

In [108]:
sent = "Probleemide korral saatke kiri aadressil dage@ut.ee või kom@ut.ee."
address_pattern = "[^@ .]*@[^@ .]*\.[^@ .]*"
addresses = re.findall(address_pattern, sent)
addresses

['dage@ut.ee', 'kom@ut.ee']

In [109]:
anonymised_sent = re.sub(address_pattern, "[e-mail varjatud]", sent)
anonymised_sent

'Probleemide korral saatke kiri aadressil [e-mail varjatud] või [e-mail varjatud].'

**Kirjutage skript, mis leiab failist 'protokoll.txt' tõenäolised nimed (suure tähega algavad sõnad, mis ei paikne lause alguses), teeb nendest kahanevas järjestuses sagedusloendi ning loob failist n-ö anonümiseeritud versiooni, asendades nimed millegi muuga. Failis paikneb iga lause eraldi real. Kasutage vähemalt kahte selle ülesande all tutvustatud teekidest. (1p)**

In [105]:
import codecs

with codecs.open("protokoll.txt", "r", "utf8") as fin:
    for line in fin:
    # ...
        

SyntaxError: unexpected EOF while parsing (<ipython-input-105-21f59e749d93>, line 6)